In [1]:
import numpy as np
import pandas as pd

from beakerx.object import beakerx
from sklearn.model_selection import train_test_split

import cimcb_lite as cb

beakerx.pandas_display_table()  # by default display pandas tables as BeakerX interactive tables

print('All packages successfully loaded')

/Users/catalinagonzalez/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


BeakerxHTML(value='You need beakerx_tabledisplay to use this')

All packages successfully loaded


# 1 Load Data and Peak sheet 

In [2]:
# The path to the input file (Excel spreadsheet)
filename = 'GastricCancer_NMR.xlsx'

# Load Peak and Data tables into two variables
dataTable, peakTable = cb.utils.load_dataXL(filename, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 140 TOTAL PEAKS: 149
Done!


In [3]:
display(dataTable)

,Idx,SampleID,SampleType,Class,M1,M2,M3,M4,M5,M6,...,M140,M141,M142,M143,M144,M145,M146,M147,M148,M149
1,1,sample_1,QC,QC,90.1,491.6,202.9,35.0,164.2,19.7,...,115.1,64.8,25.5,473.9,26.5,NaN,6.8,118.6,710.6,203.6
2,2,sample_2,Sample,GC,43.0,525.7,130.2,NaN,694.5,114.5,...,84.2,357.1,16.1,455.5,29.5,28.1,35.8,316.1,390.7,199.0
3,3,sample_3,Sample,BN,214.3,10703.2,104.7,46.8,483.4,152.3,...,993.5,1698.5,32.9,75.9,33.2,802.8,967.6,154.4,31.6,195.2
4,4,sample_4,Sample,HE,31.6,59.7,86.4,14.0,88.6,10.3,...,58.1,83.5,60.5,136.9,17.0,10.2,24.7,64.1,91.4,91.6
5,5,sample_5,Sample,GC,81.9,258.7,315.1,8.7,243.2,18.4,...,44.5,47.6,45.6,1441.7,35.2,0.1,22.8,135.0,322.3,254.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,sample_136,QC,QC,97.6,341.1,232.1,38.1,174.0,7.7,...,79.7,101.8,23.9,296.0,25.0,NaN,7.5,141.5,675.7,200.8
137,137,sample_137,Sample,GC,405.3,510.7,521.9,91.9,732.1,145.7,...,434.8,84.8,182.3,110.7,123.9,0.4,36.3,60.1,317.3,401.7
138,138,sample_138,Sample,BN,45.4,191.6,41.0,18.7,40.8,32.2,...,45.3,44.5,14.5,83.8,27.9,0.3,0.5,47.3,47.8,46.5
139,139,sample_139,Sample,HE,30.7,56.8,35.9,20.9,17.4,NaN,...,28.0,38.7,1.3,130.9,24.6,0.7,5.9,20.7,124.1,28.9


In [4]:
display(peakTable)

,Idx,Name,Label,Perc_missing,QC_RSD
1,1,M1,1_3-Dimethylurate,11.428571,32.208005
2,2,M2,1_6-Anhydro-β-D-glucose,0.714286,31.178028
3,3,M3,1_7-Dimethylxanthine,5.000000,34.990605
4,4,M4,1-Methylnicotinamide,8.571429,12.804201
5,5,M5,2-Aminoadipate,1.428571,9.372664
...,...,...,...,...,...
145,145,M145,uarm1,23.571429,41.406985
146,146,M146,uarm2,4.285714,34.458172
147,147,M147,β-Alanine,1.428571,27.623517
148,148,M148,π-Methylhistidine,1.428571,16.561921


# 2 Data Cleaning

In [5]:
# Create a clean peak table 

rsd = peakTable['QC_RSD']  
percMiss = peakTable['Perc_missing']  
pd.DataFrame(rsd).hist()
pd.DataFrame(percMiss).hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1a1345f470>]],
      dtype=object)

In [6]:
peakTableClean_20_10 = peakTable[(rsd < 20) & (percMiss < 10)]   
peakTableClean_10_5 = peakTable[(rsd < 10) & (percMiss < 5)]   

print("Number of peaks remaining: {}".format(len(peakTableClean_20_10)))
print("Number of peaks remaining: {}".format(len(peakTableClean_10_5)))

Number of peaks remaining: 52
Number of peaks remaining: 25


# 3 PCA- Quality Assesment

In [7]:
peaklist_20_10 = peakTableClean_20_10['Name']                   # Set peaklist to the metabolite names in the peakTableClean
X_20_10 = dataTable[peaklist_20_10].values                      # Extract X matrix from dataTable using peaklist
Xlog_20_10 = np.log10(X_20_10)                                  # Log scale (base-10)
Xscale_20_10 = cb.utils.scale(Xlog_20_10, method='auto')        # methods include auto, range, pareto, vast, and level

Xknn_20_10_3 = cb.utils.knnimpute(Xscale_20_10, k=3)              # missing value imputation (knn - 3 nearest neighbors)
Xknn_20_10_5 = cb.utils.knnimpute(Xscale_20_10, k=5)

print("Xknn: {} rows & {} columns".format(*Xknn_20_10_3.shape))

Xknn: 140 rows & 52 columns


In [8]:
peaklist_10_5 = peakTableClean_10_5['Name']                   # Set peaklist to the metabolite names in the peakTableClean
X_10_5 = dataTable[peaklist_10_5].values                      # Extract X matrix from dataTable using peaklist
Xlog_10_5 = np.log10(X_10_5)                                  # Log scale (base-10)
Xscale_10_5 = cb.utils.scale(Xlog_10_5, method='auto')        # methods include auto, range, pareto, vast, and level

Xknn_10_5_3 = cb.utils.knnimpute(Xscale_10_5, k=3)              # missing value imputation (knn - 3 nearest neighbors)
Xknn_10_5_5 = cb.utils.knnimpute(Xscale_10_5, k=5)

print("Xknn: {} rows & {} columns".format(*Xknn_20_10_3.shape))

Xknn: 140 rows & 52 columns


In [9]:
cb.plot.pca(Xknn_20_10_3,
            pcx=1,                                                  # pc for x-axis
            pcy=2,                                                  # pc for y-axis
            group_label=dataTable['SampleType'])

Loading BokehJS ...

In [10]:
cb.plot.pca(Xknn_20_10_3,
            pcx=1,                                                  # pc for x-axis
            pcy=2,                                                  # pc for y-axis
            group_label=dataTable['Class'])

Loading BokehJS ...

# 4 Univariate Statistics for comparison of Gastric Cancer (GC) vs Healthy Controls (HE)

In [11]:
# Select subset of Data for statistical comparison
dataTable2 = dataTable[(dataTable.Class == "GC") | (dataTable.Class == "HE")]  # Reduce data table only to GC and HE class members
pos_outcome = "GC" 

## Parametric test

In [12]:
# Calculate basic statistics and create a statistics table.
statsTable_20_10_param = cb.utils.univariate_2class(dataTable2,
                                        peakTableClean_20_10,
                                        group='Class',                # Column used to determine the groups
                                        posclass=pos_outcome,         # Value of posclass in the group column
                                        parametric=True)              # Set parametric = True or False

# View and check StatsTable
display(statsTable_20_10_param.sort_values(by=['TTestPvalue']))

,Idx,Name,Label,Grp0_Mean,Grp0_Mean-95CI,Grp1_Mean,Grp1_Mean-95CI,Sign,TTestStat,TTestPvalue,bhQvalue,TotalMissing,PercTotalMissing,Grp0_Missing,Grp1_Missing,ShapiroW,ShapiroPvalue,LeveneW,LevenePvalue
48,138,M138,u233,332.930769,"(167.71, 498.15)",1521.000000,"(1118.39, 1923.61)",1,-5.159999,0.000002,0.000091,1,1.205,2.5,0.000,0.770728,5.589887e-10,18.192045,0.000054
27,89,M89,N-AcetylglutamineDerivative,378.405000,"(289.7, 467.11)",966.325581,"(717.42, 1215.23)",1,-4.236836,0.000060,0.001549,0,0.000,0.0,0.000,0.758458,2.301412e-10,10.067315,0.002133
46,134,M134,u144,687.166667,"(512.67, 861.67)",1993.974419,"(1384.78, 2603.16)",1,-3.873564,0.000218,0.003146,1,1.205,2.5,0.000,0.677765,4.003797e-12,11.731688,0.000972
39,118,M118,Tropate,119.992500,"(91.12, 148.86)",337.895238,"(233.03, 442.76)",1,-3.843517,0.000242,0.003146,1,1.205,0.0,2.326,0.713623,2.359147e-11,14.059792,0.000333
1,4,M4,1-Methylnicotinamide,51.739474,"(39.35, 64.13)",26.477778,"(19.98, 32.98)",0,3.482846,0.000848,0.008816,9,10.843,5.0,16.279,0.861608,9.126937e-07,9.835944,0.002478
11,32,M32,Alanine,124.887500,"(100.74, 149.04)",237.700000,"(179.11, 296.29)",1,-3.397787,0.001056,0.009150,0,0.000,0.0,0.000,0.803849,3.846519e-09,10.550634,0.001691
15,45,M45,Citrate,4629.902500,"(3458.7, 5801.1)",2386.339535,"(1699.7, 3072.98)",0,3.294034,0.001466,0.010890,0,0.000,0.0,0.000,0.807643,4.957869e-09,9.381234,0.002975
3,7,M7,2-Furoylglycine,53.987179,"(31.17, 76.81)",118.525581,"(78.5, 158.55)",1,-2.672784,0.009114,0.059243,1,1.205,2.5,0.000,0.696827,1.009690e-11,5.909098,0.017299
49,142,M142,u43,11.632500,"(6.46, 16.81)",27.695238,"(16.57, 38.82)",1,-2.524698,0.013557,0.078329,1,1.205,0.0,2.326,0.615945,2.533461e-13,4.844210,0.030622
43,126,M126,trans-Aconitate,36.242500,"(28.24, 44.25)",76.670732,"(43.9, 109.44)",1,-2.323154,0.022744,0.118271,2,2.410,0.0,4.651,0.516279,6.681531e-15,3.621831,0.060669


In [13]:
# Calculate basic statistics and create a statistics table.
statsTable_10_5_param = cb.utils.univariate_2class(dataTable2,
                                        peakTableClean_10_5,
                                        group='Class',                # Column used to determine the groups
                                        posclass=pos_outcome,         # Value of posclass in the group column
                                        parametric=True)              # Set parametric = True or False

# View and check StatsTable
display(statsTable_10_5_param.sort_values(by=['TTestPvalue']))

,Idx,Name,Label,Grp0_Mean,Grp0_Mean-95CI,Grp1_Mean,Grp1_Mean-95CI,Sign,TTestStat,TTestPvalue,bhQvalue,TotalMissing,PercTotalMissing,Grp0_Missing,Grp1_Missing,ShapiroW,ShapiroPvalue,LeveneW,LevenePvalue
22,138,M138,u233,332.930769,"(167.71, 498.15)",1521.000000,"(1118.39, 1923.61)",1,-5.159999,0.000002,0.000044,1,1.205,2.5,0.000,0.770728,5.589887e-10,18.192045,0.000054
14,89,M89,N-AcetylglutamineDerivative,378.405000,"(289.7, 467.11)",966.325581,"(717.42, 1215.23)",1,-4.236836,0.000060,0.000745,0,0.000,0.0,0.000,0.758458,2.301412e-10,10.067315,0.002133
6,32,M32,Alanine,124.887500,"(100.74, 149.04)",237.700000,"(179.11, 296.29)",1,-3.397787,0.001056,0.008798,0,0.000,0.0,0.000,0.803849,3.846519e-09,10.550634,0.001691
8,45,M45,Citrate,4629.902500,"(3458.7, 5801.1)",2386.339535,"(1699.7, 3072.98)",0,3.294034,0.001466,0.009162,0,0.000,0.0,0.000,0.807643,4.957869e-09,9.381234,0.002975
2,7,M7,2-Furoylglycine,53.987179,"(31.17, 76.81)",118.525581,"(78.5, 158.55)",1,-2.672784,0.009114,0.045572,1,1.205,2.5,0.000,0.696827,1.009690e-11,5.909098,0.017299
23,142,M142,u43,11.632500,"(6.46, 16.81)",27.695238,"(16.57, 38.82)",1,-2.524698,0.013557,0.056487,1,1.205,0.0,2.326,0.615945,2.533461e-13,4.844210,0.030622
19,126,M126,trans-Aconitate,36.242500,"(28.24, 44.25)",76.670732,"(43.9, 109.44)",1,-2.323154,0.022744,0.081230,2,2.410,0.0,4.651,0.516279,6.681531e-15,3.621831,0.060669
3,8,M8,2-Hydroxyisobutyrate,79.267500,"(59.69, 98.85)",54.395349,"(42.53, 66.26)",0,2.163519,0.033448,0.104526,0,0.000,0.0,0.000,0.817766,9.915664e-09,3.216053,0.076652
24,144,M144,u87,28.280000,"(26.31, 30.25)",39.946512,"(28.79, 51.11)",1,-1.949558,0.054689,0.149324,0,0.000,0.0,0.000,0.376480,4.616048e-17,3.812503,0.054326
12,73,M73,Indole-3-lactate,76.497500,"(59.03, 93.96)",103.772093,"(82.21, 125.33)",1,-1.909565,0.059730,0.149324,0,0.000,0.0,0.000,0.907167,1.793880e-05,1.846147,0.178004


## Non parametric test

In [14]:
# Calculate basic statistics and create a statistics table.
statsTable_20_10_wil = cb.utils.univariate_2class(dataTable2,
                                        peakTableClean_20_10,
                                        group='Class',                # Column used to determine the groups
                                        posclass=pos_outcome,         # Value of posclass in the group column
                                        parametric=False)              # Set parametric = True or False

# View and check StatsTable
display(statsTable_20_10_wil.sort_values(by=['MannWhitneyPvalue']))

,Idx,Name,Label,Grp0_Median,Grp0_Median-95CI,Grp1_Median,Grp1_Median-95CI,MedianFC,Sign,MannWhitneyU,MannWhitneyPvalue,bhQvalue,TotalMissing,PercTotalMissing,Grp0_Missing,Grp1_Missing,ShapiroW,ShapiroPvalue,LeveneW,LevenePvalue
48,138,M138,u233,117.10,"(81.48, 196.3)",1178.80,"(784.8, 1765.7)",10.066610,1,254.0,5.877806e-08,0.000003,1,1.205,2.5,0.000,0.770728,5.589887e-10,18.192045,0.000054
27,89,M89,N-AcetylglutamineDerivative,283.95,"(199.9, 435.55)",700.30,"(453.84, 1027.1)",2.466279,1,403.0,3.177913e-05,0.000826,0,0.000,0.0,0.000,0.758458,2.301412e-10,10.067315,0.002133
46,134,M134,u144,523.70,"(388.66, 800.86)",1169.80,"(722.0, 1720.0)",2.233722,1,428.0,1.406853e-04,0.002439,1,1.205,2.5,0.000,0.677765,4.003797e-12,11.731688,0.000972
39,118,M118,Tropate,82.85,"(60.0, 137.81)",219.80,"(153.3, 335.41)",2.652987,1,490.0,1.185973e-03,0.015418,1,1.205,0.0,2.326,0.713623,2.359147e-11,14.059792,0.000333
15,45,M45,Citrate,3784.70,"(2610.25, 5247.15)",1883.30,"(1277.68, 2192.0)",0.497609,0,1197.0,2.164332e-03,0.022509,0,0.000,0.0,0.000,0.807643,4.957869e-09,9.381234,0.002975
49,142,M142,u43,6.10,"(3.8, 9.68)",14.55,"(9.67, 23.7)",2.385246,1,518.5,2.901147e-03,0.025143,1,1.205,0.0,2.326,0.615945,2.533461e-13,4.844210,0.030622
1,4,M4,1-Methylnicotinamide,45.95,"(25.08, 58.35)",23.05,"(14.4, 27.7)",0.501632,0,949.0,4.229112e-03,0.028487,9,10.843,5.0,16.279,0.861608,9.126937e-07,9.835944,0.002478
3,7,M7,2-Furoylglycine,26.20,"(17.1, 37.87)",62.90,"(42.4, 78.9)",2.400763,1,535.0,4.901039e-03,0.028487,1,1.205,2.5,0.000,0.696827,1.009690e-11,5.909098,0.017299
11,32,M32,Alanine,110.25,"(84.95, 147.94)",171.20,"(119.06, 229.89)",1.552834,1,551.0,4.930502e-03,0.028487,0,0.000,0.0,0.000,0.803849,3.846519e-09,10.550634,0.001691
41,120,M120,Tyrosine,40.30,"(28.0, 64.65)",77.55,"(56.8, 121.03)",1.924318,1,500.0,1.044023e-02,0.054289,5,6.024,10.0,2.326,0.591539,2.110843e-13,0.000126,0.991069


In [15]:
# Calculate basic statistics and create a statistics table.
statsTable_10_5_wil = cb.utils.univariate_2class(dataTable2,
                                        peakTableClean_10_5,
                                        group='Class',                # Column used to determine the groups
                                        posclass=pos_outcome,         # Value of posclass in the group column
                                        parametric=False)              # Set parametric = True or False

# View and check StatsTable
display(statsTable_10_5_wil.sort_values(by=['MannWhitneyPvalue']))

,Idx,Name,Label,Grp0_Median,Grp0_Median-95CI,Grp1_Median,Grp1_Median-95CI,MedianFC,Sign,MannWhitneyU,MannWhitneyPvalue,bhQvalue,TotalMissing,PercTotalMissing,Grp0_Missing,Grp1_Missing,ShapiroW,ShapiroPvalue,LeveneW,LevenePvalue
22,138,M138,u233,117.10,"(86.03, 196.3)",1178.80,"(784.8, 1792.89)",10.066610,1,254.0,5.877806e-08,0.000001,1,1.205,2.5,0.000,0.770728,5.589887e-10,18.192045,0.000054
14,89,M89,N-AcetylglutamineDerivative,283.95,"(204.02, 452.54)",700.30,"(515.81, 1027.1)",2.466279,1,403.0,3.177913e-05,0.000397,0,0.000,0.0,0.000,0.758458,2.301412e-10,10.067315,0.002133
8,45,M45,Citrate,3784.70,"(2233.64, 5249.96)",1883.30,"(1458.99, 2234.8)",0.497609,0,1197.0,2.164332e-03,0.018036,0,0.000,0.0,0.000,0.807643,4.957869e-09,9.381234,0.002975
23,142,M142,u43,6.10,"(3.92, 9.17)",14.55,"(9.07, 23.7)",2.385246,1,518.5,2.901147e-03,0.018132,1,1.205,0.0,2.326,0.615945,2.533461e-13,4.844210,0.030622
2,7,M7,2-Furoylglycine,26.20,"(21.16, 31.98)",62.90,"(40.2, 92.26)",2.400763,1,535.0,4.901039e-03,0.020544,1,1.205,2.5,0.000,0.696827,1.009690e-11,5.909098,0.017299
6,32,M32,Alanine,110.25,"(82.9, 149.75)",171.20,"(123.7, 254.8)",1.552834,1,551.0,4.930502e-03,0.020544,0,0.000,0.0,0.000,0.803849,3.846519e-09,10.550634,0.001691
19,126,M126,trans-Aconitate,32.70,"(21.59, 47.06)",49.40,"(34.2, 61.78)",1.510703,1,601.5,3.946278e-02,0.140939,2,2.410,0.0,4.651,0.516279,6.681531e-15,3.621831,0.060669
3,8,M8,2-Hydroxyisobutyrate,60.00,"(49.66, 89.87)",43.60,"(34.3, 55.1)",0.726667,0,1076.5,4.900314e-02,0.153135,0,0.000,0.0,0.000,0.817766,9.915664e-09,3.216053,0.076652
12,73,M73,Indole-3-lactate,52.80,"(41.19, 89.2)",93.10,"(62.76, 119.09)",1.763258,1,674.5,9.178830e-02,0.254967,0,0.000,0.0,0.000,0.907167,1.793880e-05,1.846147,0.178004
4,14,M14,3-Hydroxyisobutyrate,62.05,"(40.73, 87.37)",44.80,"(34.89, 54.86)",0.721998,0,977.5,1.380516e-01,0.342677,2,2.410,0.0,4.651,0.730224,6.725048e-11,1.852628,0.177348


In [16]:
# Save StatsTable to Excel
with pd.ExcelWriter("stats_GC_HE.xlsx") as writer:
    statsTable_20_10_param.sort_values(by=['TTestPvalue']).to_excel(writer, sheet_name='StatsTable_20_10_parametric', index=False)
    statsTable_10_5_param.sort_values(by=['TTestPvalue']).to_excel(writer, sheet_name='StatsTable_10_5_parametric', index=False)

    statsTable_20_10_wil.sort_values(by=['MannWhitneyPvalue']).to_excel(writer, sheet_name='StatsTable_20_10_wil', index=False)
    statsTable_10_5_wil.sort_values(by=['MannWhitneyPvalue']).to_excel(writer, sheet_name='StatsTable_10_5_wil', index=False)
print("done!")

done!


# 6 Machine Learning

## 6.1 Splitting data into Training and Test sets

In [17]:
# Create a Binary Y vector for stratifiying the samples
outcomes = dataTable2['Class']                                  # Column that corresponds to Y class (should be 2 groups)
Y = [1 if outcome == 'GC' else 0 for outcome in outcomes]       # Change Y into binary (GC = 1, HE = 0)  
Y = np.array(Y)                                                 # convert boolean list into to a numpy array

# Split DataTable2 and Y into train and test (with stratification)
dataTrain, dataTest, Ytrain, Ytest = train_test_split(dataTable2, Y, test_size=0.25, stratify=Y, random_state=10)

print("DataTrain = {} samples with {} postive cases.".format(len(Ytrain),sum(Ytrain)))
print("DataTest = {} samples with {} postive cases.".format(len(Ytest),sum(Ytest)))

DataTrain = 62 samples with 32 postive cases.
DataTest = 21 samples with 11 postive cases.


## 6.2. Determine optimal number of components for PLS-DA mode

In [18]:
# Extract and scale the metabolite data from the dataTable
XT_20_10 = dataTrain[peaklist_20_10]                                    # Extract X matrix from DataTrain using peaklist
XTlog_20_10 = np.log(XT_20_10)                                          # Log scale (base-10)
XTscale_20_10 = cb.utils.scale(XTlog_20_10, method='auto')              # methods include auto, pareto, vast, and level
XTknn_20_10_3 = cb.utils.knnimpute(XTscale_20_10, k=3)                    # missing value imputation (knn - 3 nearest neighbors)
XTknn_20_10_5 = cb.utils.knnimpute(XTscale_20_10, k=5)                    # missing value imputation (knn - 5 nearest neighbors)

In [19]:
# initalise cross_val kfold (stratified) 
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                   # model; we are using the PLS_SIMPLS model
                        X=XTknn_20_10_3,                                 
                        Y=Ytrain,                               
                        param_dict={'n_components': [1,2,3,4,5,6]},  # The numbers of latent variables to search                
                        folds=5,                                     # folds; for the number of splits (k-fold)
                        bootnum=100)                                 # num bootstraps for the Confidence Intervals


cv.run()  # run the cross validation
cv.plot() # plot cross validation statistics

Kfold: 100%|██████████| 100/100 [00:07<00:00, 13.98it/s]


Loading BokehJS ...

In [20]:
# initalise cross_val kfold (stratified) 
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                   # model; we are using the PLS_SIMPLS model
                        X=XTknn_20_10_5,                                 
                        Y=Ytrain,                               
                        param_dict={'n_components': [1,2,3,4,5,6]},  # The numbers of latent variables to search                
                        folds=5,                                     # folds; for the number of splits (k-fold)
                        bootnum=100)                                 # num bootstraps for the Confidence Intervals


cv.run()  # run the cross validation
cv.plot() # plot cross validation statistics

Kfold: 100%|██████████| 100/100 [00:07<00:00, 14.05it/s]


Loading BokehJS ...

In [21]:
# Extract and scale the metabolite data from the dataTable
XT_10_5 = dataTrain[peaklist_10_5]                                    # Extract X matrix from DataTrain using peaklist
XTlog_10_5 = np.log(XT_10_5)                                          # Log scale (base-10)
XTscale_10_5 = cb.utils.scale(XTlog_10_5, method='auto')              # methods include auto, pareto, vast, and level


XTknn_10_5_3 = cb.utils.knnimpute(XTscale_10_5, k=3)                    # missing value imputation (knn - 3 nearest neighbors)
XTknn_10_5_5 = cb.utils.knnimpute(XTscale_10_5, k=5)                    # missing value imputation (knn - 5 nearest neighbors)

In [22]:
# initalise cross_val kfold (stratified) 
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                   # model; we are using the PLS_SIMPLS model
                        X=XTknn_10_5_3,                                 
                        Y=Ytrain,                               
                        param_dict={'n_components': [1,2,3,4,5,6]},  # The numbers of latent variables to search                
                        folds=5,                                     # folds; for the number of splits (k-fold)
                        bootnum=100)                                 # num bootstraps for the Confidence Intervals


cv.run()  # run the cross validation
cv.plot() # plot cross validation statistics

Kfold: 100%|██████████| 100/100 [00:06<00:00, 14.46it/s]


Loading BokehJS ...

In [23]:
# initalise cross_val kfold (stratified) 
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                   # model; we are using the PLS_SIMPLS model
                        X=XTknn_10_5_5,                                 
                        Y=Ytrain,                               
                        param_dict={'n_components': [1,2,3,4,5,6]},  # The numbers of latent variables to search                
                        folds=5,                                     # folds; for the number of splits (k-fold)
                        bootnum=100)                                 # num bootstraps for the Confidence Intervals


cv.run()  # run the cross validation
cv.plot() # plot cross validation statistics

Kfold: 100%|██████████| 100/100 [00:06<00:00, 14.51it/s]


Loading BokehJS ...

## 6.3 Train and evaluate PLS-DA model

Le nombre de composantes qui nous donne le meilleur compromis entre sensibilité est précision est 2. Comme nous voulons faire de l'exploration sur les données nous allons choisir ce compromis au lieu d

In [24]:
modelPLS_20_10_3 = cb.model.PLS_SIMPLS(n_components=2)
modelPLS_20_10_5 = cb.model.PLS_SIMPLS(n_components=2)
modelPLS_10_5_3 = cb.model.PLS_SIMPLS(n_components=2)
modelPLS_10_5_5 = cb.model.PLS_SIMPLS(n_components=2)

In [25]:
Ypred_20_10_3 = modelPLS_20_10_3.train(XTknn_20_10_3, Ytrain)
Ypred_20_10_5 = modelPLS_20_10_5.train(XTknn_20_10_5, Ytrain)
Ypred_10_5_3 = modelPLS_10_5_3.train(XTknn_10_5_3, Ytrain)
Ypred_10_5_5 = modelPLS_10_5_5.train(XTknn_10_5_5, Ytrain)

In [26]:
modelPLS_20_10_3.evaluate(cutoffscore=0.5) 

Loading BokehJS ...

In [27]:
modelPLS_20_10_5.evaluate(cutoffscore=0.5)  

Loading BokehJS ...

In [28]:
modelPLS_10_5_3.evaluate(cutoffscore=0.5) 

Loading BokehJS ...

In [29]:
modelPLS_10_5_5.evaluate(cutoffscore=0.5)

Loading BokehJS ...

In [30]:
modelPLS_20_10_3.permutation_test(nperm=100)

Permutation Resample: 100%|██████████| 100/100 [00:01<00:00, 51.19it/s]


Loading BokehJS ...

In [31]:
modelPLS_20_10_5.permutation_test(nperm=100)

Permutation Resample: 100%|██████████| 100/100 [00:01<00:00, 53.91it/s]


Loading BokehJS ...

In [32]:
modelPLS_10_5_3.permutation_test(nperm=100)

Permutation Resample: 100%|██████████| 100/100 [00:01<00:00, 59.02it/s]


Loading BokehJS ...

In [33]:
modelPLS_10_5_5.permutation_test(nperm=100)

Permutation Resample: 100%|██████████| 100/100 [00:01<00:00, 58.97it/s]


Loading BokehJS ...

## 6.4. Plot latent variable projections for PLS-DA model

In [34]:
modelPLS_20_10_3.plot_projections(label=
                          dataTrain[['Idx','SampleID']], size=12)

Loading BokehJS ...

In [35]:
modelPLS_20_10_5.plot_projections(label=
                          dataTrain[['Idx','SampleID']], size=12) 

Loading BokehJS ...

In [36]:
modelPLS_10_5_3.plot_projections(label=
                          dataTrain[['Idx','SampleID']], size=12) 

Loading BokehJS ...

In [37]:
modelPLS_10_5_5.plot_projections(label=
                          dataTrain[['Idx','SampleID']], size=12) 

Loading BokehJS ...

## 6.5. Plot feature importance (Coefficient plot and VIP) for PLS-DA model

### 20 - 10 - 3 
#### bca

In [38]:
# Calculate the bootstrapped confidence intervals 
modelPLS_20_10_3.calc_bootci(type='bca', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_20_10_3_bca = modelPLS_20_10_3.plot_featureimportance(peakTableClean_20_10,
                                            peaklist_20_10,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Jackknife Resample: 100%|██████████| 62/62 [00:00<00:00, 842.39it/s]


Loading BokehJS ...

#### perc

In [39]:
# Calculate the bootstrapped confidence intervals 
modelPLS_20_10_3.calc_bootci(type='perc', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_20_10_3_perc = modelPLS_20_10_3.plot_featureimportance(peakTableClean_20_10,
                                            peaklist_20_10,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Bootstrap Resample: 100%|██████████| 200/200 [00:00<00:00, 707.53it/s]


Loading BokehJS ...

#### bc

In [40]:
# Calculate the bootstrapped confidence intervals 
modelPLS_20_10_3.calc_bootci(type='bc', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_20_10_3_bc = modelPLS_20_10_3.plot_featureimportance(peakTableClean_20_10,
                                            peaklist_20_10,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Bootstrap Resample: 100%|██████████| 200/200 [00:00<00:00, 361.06it/s]


Loading BokehJS ...

### 20-10-5
#### bca

In [41]:
# Calculate the bootstrapped confidence intervals 
modelPLS_20_10_5.calc_bootci(type='bca', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_20_10_5_bca = modelPLS_20_10_5.plot_featureimportance(peakTableClean_20_10,
                                            peaklist_20_10,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Jackknife Resample: 100%|██████████| 62/62 [00:00<00:00, 830.33it/s]


Loading BokehJS ...

#### perc

In [42]:
# Calculate the bootstrapped confidence intervals 
modelPLS_20_10_5.calc_bootci(type='perc', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_20_10_5_perc = modelPLS_20_10_5.plot_featureimportance(peakTableClean_20_10,
                                            peaklist_20_10,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Bootstrap Resample: 100%|██████████| 200/200 [00:00<00:00, 552.59it/s]


Loading BokehJS ...

#### bc

In [43]:
# Calculate the bootstrapped confidence intervals 
modelPLS_20_10_5.calc_bootci(type='bc', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_20_10_5_bc = modelPLS_20_10_5.plot_featureimportance(peakTableClean_20_10,
                                            peaklist_20_10,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Bootstrap Resample: 100%|██████████| 200/200 [00:00<00:00, 765.77it/s]


Loading BokehJS ...

### 10-5-3
#### bca

In [44]:
# Calculate the bootstrapped confidence intervals 
modelPLS_10_5_3.calc_bootci(type='bca', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_10_5_3_bca = modelPLS_10_5_3.plot_featureimportance(peakTableClean_10_5,
                                            peaklist_10_5,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Jackknife Resample: 100%|██████████| 62/62 [00:00<00:00, 719.76it/s]


Loading BokehJS ...

#### perc

In [45]:
# Calculate the bootstrapped confidence intervals 
modelPLS_10_5_3.calc_bootci(type='perc', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_10_5_3_perc = modelPLS_10_5_3.plot_featureimportance(peakTableClean_10_5,
                                            peaklist_10_5,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Bootstrap Resample: 100%|██████████| 200/200 [00:00<00:00, 760.05it/s]


Loading BokehJS ...

#### bc

In [46]:
# Calculate the bootstrapped confidence intervals 
modelPLS_10_5_3.calc_bootci(type='bc', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_10_5_3_bc = modelPLS_10_5_3.plot_featureimportance(peakTableClean_10_5,
                                            peaklist_10_5,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Bootstrap Resample: 100%|██████████| 200/200 [00:00<00:00, 763.33it/s]


Loading BokehJS ...

### 10-5-5
#### bca

In [47]:
# Calculate the bootstrapped confidence intervals 
modelPLS_10_5_5.calc_bootci(type='bca', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_10_5_5_bca = modelPLS_10_5_5.plot_featureimportance(peakTableClean_10_5,
                                            peaklist_10_5,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Jackknife Resample: 100%|██████████| 62/62 [00:00<00:00, 619.81it/s]


Loading BokehJS ...

#### perc

In [48]:
# Calculate the bootstrapped confidence intervals 
modelPLS_10_5_3.calc_bootci(type='perc', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_10_5_5_perc = modelPLS_10_5_3.plot_featureimportance(peakTableClean_10_5,
                                            peaklist_10_5,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Bootstrap Resample: 100%|██████████| 200/200 [00:00<00:00, 800.67it/s]


Loading BokehJS ...

#### bc

In [49]:
# Calculate the bootstrapped confidence intervals 
modelPLS_10_5_3.calc_bootci(type='bc', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet_10_5_5_bc = modelPLS_10_5_3.plot_featureimportance(peakTableClean_10_5,
                                            peaklist_10_5,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Bootstrap Resample: 100%|██████████| 200/200 [00:00<00:00, 777.31it/s]


Loading BokehJS ...

## 6.6. Test model with new data (using test set from section 6.1)

In [50]:
# Get mu and sigma from the training dataset to use for the Xtest scaling
mu_20_10, sigma_20_10  = cb.utils.scale(XTlog_20_10,  return_mu_sigma=True)
mu_10_5, sigma_10_5  = cb.utils.scale(XTlog_10_5,  return_mu_sigma=True)

In [51]:
# Pull of Xtest from DataTest using peaklist ('Name' column in PeakTable)
XV_20_10 = dataTest[peaklist_20_10].values


# Log transform, unit-scale and knn-impute missing values for Xtest
XVlog_20_10 = np.log(XV_20_10)
XVscale_20_10  = cb.utils.scale(XVlog_20_10, method='auto', mu=mu_20_10, sigma=sigma_20_10) 
XVknn_20_10_3 = cb.utils.knnimpute(XVscale_20_10, k=3)
XVknn_20_10_5 = cb.utils.knnimpute(XVscale_20_10, k=5)

In [52]:
# Pull of Xtest from DataTest using peaklist ('Name' column in PeakTable)
XV_10_5 = dataTest[peaklist_10_5].values

# Log transform, unit-scale and knn-impute missing values for Xtest
XVlog_10_5 = np.log(XV_10_5)
XVscale_10_5  = cb.utils.scale(XVlog_10_5, method='auto', mu=mu_10_5, sigma=sigma_10_5) 
XVknn_10_5_3 = cb.utils.knnimpute(XVscale_10_5, k=3)
XVknn_10_5_5 = cb.utils.knnimpute(XVscale_10_5, k=5)

In [53]:
# Calculate Ypredicted score using modelPLS.test
YVpred_20_10_3 = modelPLS_20_10_3.test(XVknn_20_10_3)

# Evaluate Ypred against Ytest
evals_20_10_3 = [Ytest, YVpred_20_10_3]    # alternative formats: (Ytest, Ypred) or np.array([Ytest, Ypred])

modelPLS_20_10_3.evaluate(evals_20_10_3, cutoffscore=0.5) 

Loading BokehJS ...

In [54]:
# Calculate Ypredicted score using modelPLS.test
YVpred_20_10_5 = modelPLS_20_10_5.test(XVknn_20_10_5)

# Evaluate Ypred against Ytest
evals_20_10_5 = [Ytest, YVpred_20_10_5]    # alternative formats: (Ytest, Ypred) or np.array([Ytest, Ypred])

modelPLS_20_10_5.evaluate(evals_20_10_5, cutoffscore=0.5) 

Loading BokehJS ...

In [55]:
# Calculate Ypredicted score using modelPLS.test
YVpred_10_5_3 = modelPLS_10_5_3.test(XVknn_10_5_3)

# Evaluate Ypred against Ytest
evals_10_5_3 = [Ytest, YVpred_10_5_3]    # alternative formats: (Ytest, Ypred) or np.array([Ytest, Ypred])

modelPLS_10_5_3.evaluate(evals_10_5_3, cutoffscore=0.5) 

Loading BokehJS ...

In [56]:
# Calculate Ypredicted score using modelPLS.test
YVpred_10_5_5 = modelPLS_10_5_5.test(XVknn_10_5_5)

# Evaluate Ypred against Ytest
evals_10_5_5 = [Ytest, YVpred_10_5_5]    # alternative formats: (Ytest, Ypred) or np.array([Ytest, Ypred])

modelPLS_10_5_5.evaluate(evals_10_5_5, cutoffscore=0.5) 

Loading BokehJS ...

## 6.7. Export results to Excel

In [57]:
# Save DataSheet as 'Idx', 'SampleID', and 'Class' from DataTest
dataSheet_20_10_3 = dataTest[["Idx", "SampleID", "Class"]].copy() 

# Add 'Ypred' to Datasheet
dataSheet_20_10_3['Ypred'] = YVpred_20_10_3

display(dataSheet_20_10_3) # View and check the dataTable 

,Idx,SampleID,Class,Ypred
4,4,sample_4,HE,0.596320
78,78,sample_78,GC,0.762340
90,90,sample_90,HE,0.190719
71,71,sample_71,GC,0.799896
92,92,sample_92,GC,1.046995
119,119,sample_119,HE,0.122116
56,56,sample_56,HE,0.166406
104,104,sample_104,HE,-0.130498
98,98,sample_98,GC,0.230983
36,36,sample_36,GC,0.401337


In [58]:
# Save DataSheet as 'Idx', 'SampleID', and 'Class' from DataTest
dataSheet_20_10_5 = dataTest[["Idx", "SampleID", "Class"]].copy() 

# Add 'Ypred' to Datasheet
dataSheet_20_10_5['Ypred'] = YVpred_20_10_5

display(dataSheet_20_10_5) # View and check the dataTable 

,Idx,SampleID,Class,Ypred
4,4,sample_4,HE,0.598554
78,78,sample_78,GC,0.781077
90,90,sample_90,HE,0.202272
71,71,sample_71,GC,0.770593
92,92,sample_92,GC,1.038851
119,119,sample_119,HE,0.130216
56,56,sample_56,HE,0.179146
104,104,sample_104,HE,-0.143722
98,98,sample_98,GC,0.243239
36,36,sample_36,GC,0.389545


In [59]:
# Save DataSheet as 'Idx', 'SampleID', and 'Class' from DataTest
dataSheet_10_5_3 = dataTest[["Idx", "SampleID", "Class"]].copy() 

# Add 'Ypred' to Datasheet
dataSheet_10_5_3['Ypred'] = YVpred_10_5_3

display(dataSheet_10_5_3) # View and check the dataTable 

,Idx,SampleID,Class,Ypred
4,4,sample_4,HE,0.658787
78,78,sample_78,GC,0.745508
90,90,sample_90,HE,-0.027369
71,71,sample_71,GC,0.959570
92,92,sample_92,GC,1.197424
119,119,sample_119,HE,0.176143
56,56,sample_56,HE,0.217196
104,104,sample_104,HE,-0.112643
98,98,sample_98,GC,0.252254
36,36,sample_36,GC,0.677516


In [60]:
# Save DataSheet as 'Idx', 'SampleID', and 'Class' from DataTest
dataSheet_10_5_5 = dataTest[["Idx", "SampleID", "Class"]].copy() 

# Add 'Ypred' to Datasheet
dataSheet_10_5_5['Ypred'] = YVpred_10_5_5

display(dataSheet_10_5_5) # View and check the dataTable 

,Idx,SampleID,Class,Ypred
4,4,sample_4,HE,0.656667
78,78,sample_78,GC,0.750820
90,90,sample_90,HE,-0.033743
71,71,sample_71,GC,0.956347
92,92,sample_92,GC,1.197956
119,119,sample_119,HE,0.202128
56,56,sample_56,HE,0.218992
104,104,sample_104,HE,-0.117132
98,98,sample_98,GC,0.252944
36,36,sample_36,GC,0.672097


In [61]:
# Create an empty excel workbook
writer = pd.ExcelWriter("modelPLS_GC_HE.xlsx")     # provide the filename for the Excel file

# Add each dataframe to the workbook in turn, as a separate worksheet
dataSheet_20_10_5.to_excel(writer, sheet_name='Datasheet 20_10_5', index=False)
peakSheet_20_10_5_bca.to_excel(writer, sheet_name='Peaksheet bca 20_10_5', index=False)
peakSheet_20_10_5_perc.to_excel(writer, sheet_name='Peaksheet perc 20_10_5', index=False)
peakSheet_20_10_5_bc.to_excel(writer, sheet_name='Peaksheet bc 20_10_5', index=False)

dataSheet_20_10_3.to_excel(writer, sheet_name='Datasheet 20_10_3', index=False)
peakSheet_20_10_3_bca.to_excel(writer, sheet_name='Peaksheet bca 20_10_3', index=False)
peakSheet_20_10_3_perc.to_excel(writer, sheet_name='Peaksheet perc 20_10_3', index=False)
peakSheet_20_10_3_bc.to_excel(writer, sheet_name='Peaksheet bc 20_10_3', index=False)

dataSheet_10_5_5.to_excel(writer, sheet_name='Datasheet 10_5_5', index=False)
peakSheet_10_5_5_bca.to_excel(writer, sheet_name='Peaksheet bca 10_5_5', index=False)
peakSheet_10_5_5_perc.to_excel(writer, sheet_name='Peaksheet perc 10_5_5', index=False)
peakSheet_10_5_5_bc.to_excel(writer, sheet_name='Peaksheet bc 10_5_5', index=False)

dataSheet_10_5_3.to_excel(writer, sheet_name='Datasheet 10_5_3', index=False)
peakSheet_10_5_3_bca.to_excel(writer, sheet_name='Peaksheet bca 10_5_3', index=False)
peakSheet_10_5_3_perc.to_excel(writer, sheet_name='Peaksheet perc 10_5_3', index=False)
peakSheet_10_5_3_bc.to_excel(writer, sheet_name='Peaksheet bc 10_5_3', index=False)


# Write the Excel workbook to disk
writer.save()

print("Done!")

Done!
